# Dowloading all the dependencies

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display
import time

max_count = 10

f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar
!python -m pip install -q --no-index --find-links=/kaggle/input/pip-download-for-langchain neural-compressor
f.value += 1
!python -m pip install -q --no-index --find-links=/kaggle/input/pip-download-for-langchain flash-attn --no-build-isolation
f.value += 1
!python -m pip install -q --no-index --find-links=/kaggle/input/pip-download-for-langchain bitsandbytes>=0.39.0 accelerate>=0.20.0
f.value += 1
!python -m pip install -q --no-index --find-links=/kaggle/input/pip-download-for-langchain langchain
f.value += 1
!python -m pip install -q --no-index --find-links=/kaggle/input/pip-download-for-langchain pypdf
f.value += 1
!python -m pip install -q --no-index --find-links=/kaggle/input/pip-download-for-langchain sentence-transformers
f.value += 1
!pip install -q faiss-gpu
f.value += 1
!pip install -q indic-nlp-library
f.value += 1
!pip install -q -U sacremoses
f.value += 1
!pip install -q -U google-generativeai
f.value += 1
!pip install -q urduhack
f.value +=1

IntProgress(value=0, max=10)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.3 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.12.0 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.12.0 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, but you have dask 2023.12.0 which is incompatible.
dask-cuda 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.12.0 which is incompatible.
dask-cuda 23.8.0 requires pandas<1.6.0dev0,

In [ ]:
import git

# Clone the repository from the specified URL to the target directory
repo_url = 'https://github.com/VarunGumma/IndicTransTokenizer'
target_directory = '/kaggle/working/sample'

# Clone the repository
git.Repo.clone_from(repo_url, target_directory)

In [ ]:
import sys
# Append the specified directory to the Python sys.path
target_directory = '/kaggle/working/sample'
sys.path.append(target_directory)

In [ ]:
# Import required modules and classes
from transformers import BitsAndBytesConfig  # For quantization of the LLM model
import torch
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer # For initialization of the translational model
from transformers import AutoModelForSeq2SeqLM # For initialization of the translational model
import transformers  # For the initialization of the LLM Model
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

# Import modules for vector database
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate  # Helps in creating a prompt template
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import TextLoader

# Additional imports
import torch
import torchaudio

In [ ]:
from transformers import BitsAndBytesConfig
# Create a quantization configuration using BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable 4-bit quantization during model loading
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Initializing Automatic speech recognition using IndicWav2Vec by AI4Bharat

In [ ]:
from transformers import pipeline
transcriber = pipeline("automatic-speech-recognition", model="ai4bharat/indicwav2vec-hindi", device="cuda")

# Initialiazation of the AI4Bharat IndicTrans2
### This model has amazing translational capabilities which will be helpful to us to answer a wide diversity of people

### Indic to English

In [ ]:
# Instantiate an IndicTransTokenizer for transliterating from Indic to English
tokenizer_indic_to_en = IndicTransTokenizer(direction="indic-en")

# Instantiate an IndicProcessor for processing transliteration from Indic to English
ip_indic_to_en = IndicProcessor(inference=True)

# Load a pre-trained model for transliteration from Indic to English using AutoModelForSeq2SeqLM
model_indic_to_en = AutoModelForSeq2SeqLM.from_pretrained(
    "ai4bharat/indictrans2-indic-en-1B",
    trust_remote_code=True,
    device_map="cuda:0",
    quantization_config=quantization_config  # Quantization configuration for the model
)

### English to Indic

In [ ]:
# Load the tokenizer using IndicTransTokenizer for transliterating from English to Indic
tokenizer_en_to_indic = IndicTransTokenizer(direction="en-indic")

# Instantiate an IndicProcessor for processing transliteration from English to Indic
ip_en_to_indic = IndicProcessor(inference=True)

# Load a pre-trained model for transliteration from English to Indic using AutoModelForSeq2SeqLM
model_en_to_indic = AutoModelForSeq2SeqLM.from_pretrained(
    "ai4bharat/indictrans2-en-indic-dist-200M",
    trust_remote_code=True,
    device_map="cuda:0"
)

# Initialization of the LLM Model - Intel Neural Chat v-3.3

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define the model name
model_name = 'Intel/neural-chat-7b-v3-3-Slerp'

# Load a pre-trained model for causal language modeling using AutoModelForCausalLM
neural_intel_chat_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda:0",
    quantization_config=quantization_config  # Quantization configuration for the model
)

# Load the corresponding tokenizer using AutoTokenizer
neural_intel_chat_tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
pipe = transformers.pipeline("text-generation", model=neural_intel_chat_model, tokenizer=neural_intel_chat_tokenizer, max_new_tokens=256)
hf = HuggingFacePipeline(pipeline=pipe)

# Initialization of Gemini API

In [ ]:
import pathlib
import textwrap

# Import the generative AI module from the google package
import google.generativeai as genai

# Import necessary display modules from IPython
from IPython.display import display, Markdown

def to_markdown(text):
    """
    Convert plain text to Markdown format.

    This function takes a plain text input and converts it to Markdown format.
    It also replaces bullet points with proper Markdown list syntax.

    Args:
        text (str): The plain text to be converted to Markdown.

    Returns:
        Markdown: The converted Markdown text.

    Example:
        >>> plain_text = "This is a bullet point:\n• Item 1\n• Item 2"
        >>> markdown_output = to_markdown(plain_text)
        >>> print(markdown_output)
        > This is a bullet point:
        >   * Item 1
        >   * Item 2
    """
    # Replace special character '•' with proper Markdown list syntax
    text = text.replace('•', '  *')
    
    # Indent the text with '>' to format it as a blockquote
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
from kaggle_secrets import UserSecretsClient

# Create a UserSecretsClient instance to access Kaggle user secrets
user_secrets = UserSecretsClient()

# Retrieve the Google API key from Kaggle user secrets
# Note: Ensure that you have previously stored the "Google API key" as a secret on Kaggle
secret_value_0 = user_secrets.get_secret("sif")

# Configure the generative AI module with the obtained Google API key
genai.configure(api_key=secret_value_0)

In [ ]:
# Create a generative language model instance using the 'gemini-pro' model
llm_model = genai.GenerativeModel('gemini-pro')

# Initialization of TTS Hindi

In [ ]:
from transformers import VitsModel, AutoTokenizer
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
tts_hindi_model = VitsModel.from_pretrained("facebook/mms-tts-hin").to(device)
tts_hindi_tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-hin")

# Initialization of the Vector Database

In [ ]:
# Define the model name for Hugging Face embeddings
model_name = 'WhereIsAI/UAE-Large-V1'

# Define model kwargs for Hugging Face embeddings during initialization
model_kwargs = {'device': 'cuda:0'}

# Define encode kwargs for Hugging Face embeddings
# Set 'normalize_embeddings' to True to compute cosine similarity
encode_kwargs = {'normalize_embeddings': True}

In [ ]:
# Create a TextLoader instance for loading text documents
# Specify the path to the text file ("/kaggle/input/evspdf/RAG.txt" in this case)
loader = TextLoader("/kaggle/input/rag-dataset-bhuvan/RAGlatestDatabaseNew.txt")

# Load documents using the TextLoader
documents = loader.load()

In [ ]:
# Create a RecursiveCharacterTextSplitter instance for splitting text documents
# Set a small chunk size and overlap for demonstration purposes
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False
)

# Split the loaded documents using the TextSplitter
text_docs = text_splitter.split_documents(documents)

# Create Hugging Face embeddings instance
# Specify the model name, model kwargs, and encode kwargs
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Create a FAISS vector store from the split text documents and embeddings
db = FAISS.from_documents(text_docs, embeddings)

# Conversation History Tools

In [ ]:
def get_history(k=3):
    """
    Retrieve conversation history for contextual understanding.

    Parameters:
    - k (int): The number of conversation tiles to include in the context (default is 3).

    Returns:
    str: A string containing the conversation history up to the specified number of tiles.
    """

    # Initialize an empty string to store the conversation history
    conversation = ""

    # Get the length of the conversation history
    l_conv = len(conversation_history)

    # If the length of the conversation history is greater than k, retrieve the last k tiles
    if l_conv > k:
        for i in range(l_conv - k, l_conv):
            conversation += conversation_history[i]
    else:
        # If the length is less than or equal to k, include the entire conversation history
        for i in conversation_history:
            conversation += i

    # Return the concatenated conversation history string
    return conversation

# Query Processing

### Taking the query from the user

### Convert the query from Indic to English

In [ ]:
def translate_indic_to_english(user_input,source_language,target_language):
    # Record the start time for performance measurement
    begin = time.time()
    # Check if the source language is different from the target language for transliteration
    if not (source_language == target_language):
        # Preprocess the user input batch for transliteration
        batch = ip_indic_to_en.preprocess_batch([user_input], src_lang=source_language, tgt_lang=target_language)
        batch = tokenizer_indic_to_en(batch, src=True, return_tensors="pt")

        # Generate transliterations using the pre-trained model
        with torch.inference_mode():
            user_input_translated = model_indic_to_en.generate(
                batch.input_ids.to("cuda"), num_beams=5, num_return_sequences=1, max_length=256
            )

        # Decode the generated sequences and post-process the transliterations
        user_input_translated = tokenizer_indic_to_en.batch_decode(user_input_translated, src=False)
        user_input_translated = ip_indic_to_en.postprocess_batch(user_input_translated, lang=target_language)

        # Print the transliterated user input
        # print(user_input_translated)
    else:
        # If the source and target languages are the same, no transliteration is needed
        user_input_translated = [user_input]
    end = time.time()
    return user_input_translated,end-begin

### Checking the vector database for getting some relevant question from the user

In [ ]:
def get_context(user_input_translated,k=2, fetch_k=50):
    '''
    Perform max marginal relevance search using the FAISS vector store
    Search for relevant documents based on the transliterated user input
    Usually you would want the fetch_k parameter >> k parameter
    This is because the fetch_k parameter is the number of documents that will be fetched before filtering. 
    If you set fetch_k to a low number, you might not get enough documents to filter from.
    k = Number of searches to be returned
    fetch_k = Number of documents that will be fetched before filtering
    '''
    docs = db.max_marginal_relevance_search(user_input_translated[0], k=k, fetch_k=fetch_k) 
    # Initialize an empty string to store concatenated contexts
    contexts = ""

    # Iterate through the retrieved documents and print page content
    for i, doc in enumerate(docs):
        # print(f"{i + 1}. {doc.page_content}")

        # Concatenate the page content for further processing if needed
        contexts += doc.page_content + "\n\n"
    
    return contexts

# Converting English output from the Model back to Hindi

In [ ]:
def translate_english_to_indic(en_text,source_language,target_language):
    begin = time.time()
    # Translate English text to Hindi if the source and target languages are different
    if not (source_language == target_language):
        # Preprocess the English text for translation
        batch = ip_en_to_indic.preprocess_batch([en_text], src_lang=target_language, tgt_lang=source_language)
        batch = tokenizer_en_to_indic(batch, src=True, return_tensors="pt")

        # Generate translations using the pre-trained model
        with torch.inference_mode():
            outputs = model_en_to_indic.generate(
                batch.input_ids.to("cuda"), num_beams=2, num_return_sequences=1, max_length=256
            )

        # Decode the generated sequences and post-process the translations
        outputs = tokenizer_en_to_indic.batch_decode(outputs, src=False)
        outputs = ip_en_to_indic.postprocess_batch(outputs, lang=source_language)
    else:
        # If the source and target languages are the same, no translation is needed
        outputs = [en_text]

    # Record the end time for performance measurement
    end = time.time()

    # Measure and print the elapsed time for the operation
    elapsed_time = end - begin
    
    return outputs,elapsed_time

In [ ]:
conversation_history = []

In [ ]:
def conversation(file_path, language_choice, speed_choice, is_audio=False, time_it=True, print_context=False):
    """
    Conducts a conversation with the chatbot, translating user input, retrieving context, and generating responses.

    Parameters:
    - user_input (str): The input provided by the user in an Indic language.
    - time_it (bool): A flag indicating whether to print the time taken for each step (default is True).

    Returns:
    None
    """
    # Define a dictionary mapping languages to their corresponding Indic2Trans language codes
    languages = {
        "Assamese":"asm_Beng",
        "Kashmiri (Arabic)":"kas_Arab",
        "Punjabi":"pan_Guru",
        "Bengali":"ben_Beng",
        "Kashmiri (Devanagari)":"kas_Deva",
        "Sanskrit":"san_Deva",
        "Bodo":"brx_Deva",
        "Maithili":"mai_Deva",
        "Santali":"sat_Olck",
        "Dogri":"doi_Deva",
        "Malayalam":"mal_Mlym",
        "Sindhi (Arabic)":"snd_Arab",
        "Marathi":"mar_Deva",
        "English":"eng_Latn",
        "Sindhi (Devanagari)":"snd_Deva",
        "Konkani":"gom_Deva",
        "Manipuri (Bengali)":"mni_Beng",
        "Tamil":"tam_Taml",
        "Gujarati":"guj_Gujr",
        "Manipuri (Meitei)":"mni_Mtei",
        "Telugu":"tel_Telu",
        "Hindi":"hin_Deva",
        "Nepali":"npi_Deva",
        "Urdu":"urd_Arab",
        "Kannada":"kan_Knda",
        "Odia":"ory_Orya"
    }
    # Select the source language for transliteration based on the user's choice
    source_language = languages[language_choice]

    # Set the target language for transliteration to English
    target_language = "eng_Latn"
    # ASR model to convert the audio to text
    if is_audio:
        user_input = transcriber(file_path)
        user_input = user_input["text"]
    else:
        user_input = file_path

    # Translate user input from Indic to English
    user_input_translated, t1_time = translate_indic_to_english(user_input,source_language,target_language)

    # Record the start time for performance measurement
    begin = time.time()

    if speed_choice == 0:
        # Get context based on the translated user input
        context = get_context(user_input_translated,k=1)
        if print_context:
            print(context)
    else:
        # Get context based on the translated user input
        context = get_context(user_input_translated)
        if print_context:
            print(context)

    # Calculate the time taken for the vector database operation
    vectordb_time = time.time() - begin

    if speed_choice==1:
        # Create a system input for the chatbot including context and conversation history
        system_input = f"""As a helpful chatbot specialized in providing concise and relevant information about websites, your goal is to offer insightful responses to user queries. Utilize both the provided context and your chat history, ensuring the inclusion of the website mentioned in the context in your responses. Strive to address the user's inquiry in one to two sentences while maximizing assistance.
Chat History: {get_history()}
Context: {context}
User Query: {user_input_translated}
"""

        # Record the start time for LLM generation
        begin = time.time()

        # Generate an English response using the language model pipeline
        response = llm_model.generate_content(system_input)
        en_text = response.text
        print(en_text)

        # Calculate the time taken for the LLM generation
        llm_time = time.time() - begin
        
    elif speed_choice==2:
        system_input = f"""As a helpful chatbot specialized in providing concise and relevant information about websites, your goal is to offer insightful responses to user queries. Utilize the provided context, ensuring the inclusion of the website mentioned in the context in your responses. Strive to address the user's inquiry in one to two sentences while maximizing assistance.
Please remember to include meaningful links and try to help the users as much as you can.
Context: {context}
"""
        template = f"### System:\n{system_input}\n### User:\n{user_input_translated}\n### Assistant:\n"

        # Record the start time for LLM generation
        begin = time.time()

        # Generate an English response using the language model pipeline
        en_text = pipe(template)

        # Calculate the time taken for the LLM generation
        llm_time = time.time() - begin

        # Extract the generated English text from the pipeline response
        en_text = en_text[0]["generated_text"].replace(template, "")
        
    else:
        en_text = context

    # Update conversation history with user input and assistant response
    history = f"User: {user_input_translated[0]}\nAssistant: {en_text}\n"
    conversation_history.append(history)

    # Translate the generated English text back to Indic
    output, t2_time = translate_english_to_indic(en_text,source_language,target_language)
    
    if language_choice == "Hindi":
        text = output[0]
        inputs = tokenizer(text, return_tensors="pt").to(device)
        with torch.no_grad():
            audio_tensor = model(**inputs).waveform
        torchaudio.save('test_1.mp3',
                  audio_tensor.cpu(),
                  sample_rate=16000)

    # Print the translated output
    return(output[0],user_input)

# Server


In [ ]:
!pip install -q python-multipart multipart fastapi[all] nest-asyncio pyngrok uvicorn python-firebase firebase-admin python-jose passlib pydub

In [ ]:
from typing import Optional
from pydantic import BaseModel
from fastapi import FastAPI, HTTPException, Form, File, UploadFile
import firebase_admin
from firebase_admin import credentials, firestore
from fastapi import FastAPI, Form
import base64
from fastapi import FastAPI, Depends, HTTPException
from fastapi.security import OAuth2PasswordBearer
from jose import JWTError, jwt
from passlib.context import CryptContext
from datetime import datetime, timedelta
from typing import Optional
import os
from typing import Optional
from pydantic import BaseModel
from fastapi import FastAPI, HTTPException, Form
import firebase_admin
from firebase_admin import credentials, firestore
from fastapi import FastAPI, Form
from fastapi.middleware.cors import CORSMiddleware
from fastapi import HTTPException
import base64
import re
import base64
import io
import wave
import base64
import io
from pydub import AudioSegment

In [ ]:
def decode_base64_to_mp3(base64_data, output_path="/kaggle/working/decoded_audio.mp3"):
    try:
        # Decode base64 audio data
        audio_bytes = base64.b64decode(base64_data)

        # Create a file-like object from the decoded bytes
        audio_file = io.BytesIO(audio_bytes)

        # Load audio data using pydub
        audio_segment = AudioSegment.from_file(audio_file, format="wav")

        # Export the audio to an MP3 file
        audio_segment.export(output_path, format="mp3")

        print(f"Audio successfully decoded and saved to {output_path}")
        return output_path

    except Exception as e:
        print(f"Error decoding audio: {e}")


In [ ]:
app = FastAPI()

# CORS middleware to allow requests from the frontend
origins = ["*"]  # Adjust as needed
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# JWT Configuration
SECRET_KEY = "$ecret@2024!@#"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

# Dependency to get the current user from the token
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

class User(BaseModel):
    email: str
    password: str
        
class Query(BaseModel):
    query: str
    lang: str
    model: str    

class BaseAudio(BaseModel):
    base64info: str
    lang: str
    model: str
    
# Firebase user management functions
def create_user(email: str, password: str):
    user = auth.create_user_with_email_and_password(email, password)
    return user

def login_user(email: str, password: str):
    user = auth.sign_in_with_email_and_password(email, password)
    return user

@app.post("/api/register")
async def register(email: str = Form(...), password: str = Form(...)):
    try:
        # Check for existing user
        user_doc = db.collection("users").document(email).get()
        if user_doc.exists:
            return {"success": False, "message": "User already exists!"}

        # Create new user document
        user_data = {"email": email, "password": password}
        db.collection("users").document(email).set(user_data)

        return {"success": True, "message": "Registration successful"}

    except Exception as e:
        raise HTTPException(status_code=500, detail="Registration failed!")

@app.post("/api/login")
async def login(email: str = Form(...), password: str = Form(...)):
    try:
        # Retrieve user from Firestore
        user_doc = db.collection("users").document(email).get()
        user_data = user_doc.to_dict()

        # Check password
        if user_data.get("password") != password:
            raise HTTPException(status_code=401, detail="Invalid credentials!")

        # Implement authentication logic (e.g., JWT)
        return {"success": True, "message": "Login Successful"}

    except Exception as e:
        return {"success":False, "message":"User does not exist!"}

@app.post("/query")
async def querySF(req: Query):
    print(req.query)
    response, _ = conversation(req.query, speed_choice=int(req.model), language_choice=req.lang)
    ((response.replace('[bhuvan-wbis.nrsc.gov.in]','')).replace(')', ' )')).replace(']', ' ]')
    return(response)
    

@app.post("/upload-audio")
async def upload_audio(req:BaseAudio):
    try:
        decode_base64_to_mp3(req.base64info)
        # Decode base64 audio data
        audio_bytes = base64.b64decode(req.base64info)

        # Create a file-like object from the decoded bytes
        audio_file = io.BytesIO(audio_bytes)
        output_path="/kaggle/working/audio.wav"
        with wave.open(output_path, 'wb') as wf:
            wf.setnchannels(1)  # Mono audio
            wf.setsampwidth(2)  # 16-bit audio
            wf.setframerate(44100)  # Sample rate (adjust as needed)
            wf.writeframes(audio_file.read())
        print(f"Audio successfully decoded and saved to {output_path}")
        response, user_input = conversation("What is Bhuvan?", speed_choice=int(req.model), language_choice=req.lang, is_audio=False)
        print(response)
        return(response)

    except Exception as e:
        print(f"Error decoding audio: {e}")

In [ ]:
!ngrok config add-authtoken 2ax1IpRRcG4GZhHVmYVQbG0pcFV_6Yeg42KGpZ8J9RtVTzJwd

In [ ]:
import os
from typing import Optional, Dict
import uvicorn
import nest_asyncio
from pyngrok import ngrok
# specify a port
port = 8000
ngrok_tunnel = ngrok.connect(port, bind_tls=True, hostname="sif.ngrok.io")

# where we can visit our fastAPI app
print('Public URL:', ngrok_tunnel.public_url)


nest_asyncio.apply()

# finally run the app
uvicorn.run(app, port=port)

# it will take some moment (~10 sec) to load the model

# Running Samples

In [ ]:
%%time
user_input3 = "कोरोना के बारे में बताएं"
conversation(user_input3,speed_choice=2,language_choice="Hindi",print_context=True)

In [ ]:
%%time
user_input2 = "मुझे आधार केंद्र बारे में विस्तार में बताएं"
conversation(file_path=user_input2,speed_choice=2,language_choice="Hindi",print_context=True)

In [ ]:
%%time
user_input2 = "What was my previous question?"
conversation(file_path=user_input2,language_choice="English")

In [ ]:
%%time
user_input5 = "मुझे अब तक हुई हमारी सारी बातचीत का सारांश दीजिए"
conversation(user_input5,language_choice="Hindi")

In [ ]:
%%time
user_input6 = "भारत के जंगलों के बारे में बताओ"
conversation(user_input6,language_choice="Hindi",speed_choice=1)

In [ ]:
%%time
user_input7 = "मुझे भारत में वनों के बारे में बताएं?"
conversation(user_input7,language_choice="Hindi",speed_choice=1)

In [ ]:
%%time
user_input8 = "मुझे भारत में खदानों के बारे में बताएं?"
conversation_history = []
conversation(user_input8, language_choice="Hindi",speed_choice=1,print_context=True)

In [ ]:
%%time
user_input9 = "Can you tell me about water resource management and assessment"
conversation(user_input9, language_choice="English",speed_choice=2,print_context=True)

In [ ]:
%%time
user_input9 = "Can you give me the website for the technical documentation of WBIS"
conversation(user_input9, language_choice="English",speed_choice=0,print_context=True)

In [ ]:
chat.history